In [364]:
## 전처리 함수

import pandas as pd 
import numpy as np
import datetime as dt
from  datetime import datetime, timedelta, date
from math import sqrt
import time
import pickle
from itertools import chain, repeat

## display
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_seq_items', None)

# df

In [365]:
class GetData():
    def __init__(self, path):
        self.path = path
    
    def get_df_fw(self, raw = False):
    
        ## 제공 데이터 불러오기
        df_fw = pd.read_csv(self.path + '01_음식물쓰레기_FOOD_WASTE.CSV', encoding='cp949')
        
        ## 컬럼명 한글로 변경
        df_fw.columns = ['배출일자','시_구분','행정동_코드','행정동명','배출거점지역_코드','배출건수','배출량','총_지불금액']
        df_fw = df_fw[df_fw['행정동명']!='알수없음']
        df_fw['행정동_코드'] = df_fw['행정동_코드'].astype(str)

        df_fw.rename({'배출일자':'기준일자'}, axis=1, inplace=True)
        df_fw['기준일자'] = pd.to_datetime(df_fw['기준일자'])

        dff1 = df_fw.groupby(['기준일자','시_구분', '행정동_코드', '행정동명']).sum().reset_index()
        dff1 = dff1[dff1.행정동명 != "알수없음"]

        ## 결측치 만들기
        df_fw1 = df_fw[df_fw.행정동명!="알수없음"]
        df_fw1["기준일자"] = pd.to_datetime(df_fw1['기준일자'], format='%Y-%m-%d')
        unique_code= df_fw1[df_fw1.기준일자 =="2020-06-01"][["행정동명","행정동_코드"]]
        df_fw1 = df_fw1[["배출량",'기준일자',"시_구분","행정동_코드","행정동명"]] #사용할 열 선택 
        #data = data[data.행정동명!="알수없음"]

        ###그룹화 
        df_fw1 = df_fw1.groupby(['기준일자',"시_구분","행정동_코드","행정동명"]).sum().reset_index()

        ###없는 지역의 값 채우기
        date_col = df_fw1.기준일자.unique().tolist()*41
        unique_local = df_fw1.행정동명.unique().tolist()
        local_col = list(chain.from_iterable((repeat(unique_local, 1277) for unique_local in unique_local)))

        nan_add= pd.DataFrame({"기준일자":date_col,"행정동명":local_col})

        df2 = df_fw1[df_fw1.기준일자 =="2020-06-01"][["시_구분","행정동명","행정동_코드"]]
        df3 = pd.merge(df2, nan_add, how='right', on=["행정동명"])

        df3["기준일자"] = pd.to_datetime(df3['기준일자'], format='%Y-%m-%d')
        data2 = pd.merge(df_fw1, df3, how='right', on=["기준일자","행정동명","시_구분","행정동_코드"])
        unique_df = data2[['기준일자','시_구분','행정동_코드','행정동명']]

        na_list =["한경면","조천읍","한림읍","구좌읍"]
        #data3 = data2[~data2.행정동명.isin(na_list)][['기준일자',"시_구분","행정동명","배출량"]]
        data3 = data2[['기준일자',"행정동명","배출량"]]
        data3 = data3.pivot(index=["기준일자"],columns='행정동명', values='배출량').reset_index()

        data4 = data3[['기준일자','건입동','남원읍','노형동','대륜동','대정읍','대천동','도두동','동홍동','봉개동','삼도1동',
                    '삼도2동','삼양동', '서홍동', '성산읍','송산동','아라동', '안덕면','애월읍',
                    '연동','영천동','예래동','오라동','외도동','용담1동','용담2동','이도1동','이도2동','이호동',
                    '일도1동','일도2동','정방동','중문동','중앙동', '천지동','표선면', '화북동','효돈동']].set_index("기준일자")    
        data_nafill = data4.interpolate(method='time')
        data5= pd.concat([data_nafill.reset_index(),data3[na_list]],axis=1)
        df_fw_na = pd.melt(data5,id_vars=["기준일자"],value_name="배출량2")
        df_fw_na2=pd.merge(df_fw_na,unique_df,how ="inner",on=["기준일자","행정동명"])
        df_fw_na2["기준일자"] = pd.to_datetime(df_fw_na2['기준일자'], format='%Y-%m-%d')
        dff1["기준일자"] = pd.to_datetime(dff1['기준일자'], format='%Y-%m-%d')

        df_fw_na2.행정동_코드 = df_fw_na2.행정동_코드.astype("int")
        dff1.행정동_코드 = dff1.행정동_코드.astype("int")
        dff1=pd.merge(df_fw_na2,dff1,how ="left",on=["기준일자","시_구분","행정동명","행정동_코드"])
        dff1.기준일자 = dff1.기준일자.astype("str")
        df_fw = df_fw[df_fw.행정동명!="알수없음"]
        dff2 = df_fw.groupby(['기준일자', '시_구분', '행정동명']).count().reset_index()[['기준일자', '시_구분', '행정동명', '배출거점지역_코드']]
        dff2.rename({"배출거점지역_코드":"거점지역_개수"},axis=1,inplace=True)
        dff2.기준일자 =pd.to_datetime(dff2.기준일자, format='%Y-%m-%d')
        unique_df.기준일자 = pd.to_datetime(unique_df.기준일자 , format='%Y-%m-%d')
        dff2_2=pd.merge(dff2,unique_df,how="right",on=["기준일자","시_구분","행정동명"]) 
        #######################################################################
        dff2_2=dff2_2[["기준일자","행정동명","거점지역_개수"]]
        dff2_2 = dff2_2.pivot(index=["기준일자"],columns='행정동명', values='거점지역_개수').reset_index()
        dff2_3 = dff2_2[['기준일자','건입동','남원읍','노형동','대륜동','대정읍','대천동','도두동','동홍동','봉개동','삼도1동',
                        '삼도2동','삼양동', '서홍동', '성산읍','송산동','아라동', '안덕면','애월읍',
                        '연동','영천동','예래동','오라동','외도동','용담1동','용담2동','이도1동','이도2동','이호동',
                        '일도1동','일도2동','정방동','중문동','중앙동', '천지동','표선면', '화북동','효돈동']].set_index("기준일자")
        data_nafill2 = dff2_3.interpolate(method='time')
        dff2= pd.concat([data_nafill2.reset_index(),dff2_2[na_list]],axis=1)
        dff2=pd.melt(dff2,id_vars=["기준일자"],value_name="거점지역_개수")
        dff1['기준일자'] = pd.to_datetime(dff1['기준일자'])
        dfff = pd.merge(dff1, dff2, on=['기준일자','행정동명'], how='left')
        dfff = dfff.drop(['배출량'], axis=1).rename({'배출량2':'배출량'}, axis=1)
        dfff['행정동_코드'] = dfff['행정동_코드'].astype(str)
        
        
        if raw == True:
            df_fw_raw = df_fw.copy()
            del df_fw
            return df_fw_raw
        
        else:
            df_fw = dfff.copy()
            #temp1 = df_fw.groupby(['기준일자','시_구분','행정동명','행정동_코드']).count().reset_index().iloc[:,:5].rename({"배출거점지역_코드":"거점지역_개수"},axis=1)
            #df_fw_group = df_fw.groupby(['기준일자','시_구분','행정동명','행정동_코드']).sum().reset_index()
            #df_fw = pd.merge(df_fw_group, temp1, on=['기준일자','시_구분','행정동명','행정동_코드'])

            #del temp1, df_fw_group
            
            return df_fw

            
    def get_df_card(self, raw = False):    
        df_card = pd.read_csv(self.path + '04_음식관련 카드소비_CARD_SPENDING.CSV', encoding='cp949')
        df_card.columns = ['기준일자','시_구분','행정동_코드','행정동명','업종_코드','업종명','결제건수','결제금액']
        df_card = df_card.drop(['행정동_코드','업종_코드'],axis=1)
        df_card["기준일자"] = pd.to_datetime(df_card['기준일자'], format='%Y-%m-%d')
        
        if raw == True:
            df_card_raw = df_card.copy()
            del df_card
            return df_card_raw
        
        else:
            piv1 = df_card.pivot(index=['기준일자', '시_구분', '행정동명'],
                         columns='업종명',values='결제건수').reset_index()
            
            piv1.rename({'간식':'간식_결제건수', '농축수산물':'농축수산물_결제건수', '마트/슈퍼마켓':'마트/슈퍼마켓_결제건수',
             '배달':'배달_결제건수', '부페':'부페_결제건수', '식품':'식품_결제건수', '아시아음식':'아시아음식_결제건수',
             '양식':'양식_결제건수', '주점및주류판매':'주점및주류판매_결제건수',
             '패스트푸드':'패스트푸드_결제건수', '한식':'한식_결제건수'},axis=1,inplace=True)
            
            piv2 = df_card.pivot(index=['기준일자', '시_구분', '행정동명'],
                         columns='업종명',values='결제금액').reset_index()
            
            piv2.rename({'간식':'간식_결제금액', '농축수산물':'농축수산물_결제금액', '마트/슈퍼마켓':'마트/슈퍼마켓_결제금액',
             '배달':'배달_결제금액', '부페':'부페_결제금액', '식품':'식품_결제금액', '아시아음식':'아시아음식_결제금액',
             '양식':'양식_결제금액', '주점및주류판매':'주점및주류판매_결제금액',
             '패스트푸드':'패스트푸드_결제금액', '한식':'한식_결제금액'},axis=1,inplace=True)
            
            pivot_card = pd.concat([piv1, piv2.iloc[:,3:]], axis=1)
            return pivot_card

    def get_df_kor(self, raw = False):
        df_kor = pd.read_csv(self.path + '02-1_내국인유동인구_KOREAN.CSV', encoding='cp949')

        ## 컬럼명 한글로 변경
        df_kor.columns = ['기준일자','시간','시_구분','행정동_코드','행정동명','성별','연령대','거주인구','근무인구','방문인구','거주지역행정구역']
        df_kor['기준일자'] = pd.to_datetime(df_kor['기준일자'])
        df_kor['행정동_코드'] = df_kor['행정동_코드'].astype(str)

        df_kor['시간'] = df_kor['시간'].astype('int16')
        df_kor[['거주인구','근무인구','방문인구']] = df_kor[['거주인구','근무인구','방문인구']].astype('float32')
        df_kor['시간대'] = df_kor['시간'].apply(lambda x: '낮' if x in range(8,21) else '밤')

        if raw == True:
            df_kor_raw = df_kor.copy()
            del df_kor
            return df_kor_raw

        else:

            df_kor1 = df_kor.groupby(['기준일자','시간대','시_구분','행정동명','행정동_코드','연령대','거주지역행정구역']).sum().reset_index().drop(['시간'], axis=1)
            pop_거주 = df_kor1.pivot(index=["기준일자",'시간대',"시_구분","행정동명",'행정동_코드','연령대'], columns='거주지역행정구역', values='거주인구').reset_index().rename({"그외" : "그외_거주인구","제주" : "제주_거주인구"},axis=1)
            pop_근무 = df_kor1.pivot(index=["기준일자",'시간대',"시_구분","행정동명",'행정동_코드','연령대'], columns='거주지역행정구역', values='근무인구').reset_index().rename({"그외" : "그외_근무인구","제주" : "제주_근무인구"},axis=1)
            pop_방문 = df_kor1.pivot(index=["기준일자",'시간대',"시_구분","행정동명",'행정동_코드','연령대'], columns='거주지역행정구역', values='방문인구').reset_index().rename({"그외" : "그외_방문인구","제주" : "제주_방문인구"},axis=1)

            df_kor2 = pd.concat([pop_거주, pop_근무.iloc[:,6:], pop_방문.iloc[:,6:]], axis=1)

            pop1_제주_거주 = df_kor2.pivot(index=["기준일자","시_구분","행정동명",'행정동_코드','연령대'], columns='시간대', values='제주_거주인구').reset_index().rename({"낮" : "낮_제주_거주인구","밤" : "밤_제주_거주인구"},axis=1)
            pop1_그외_근무 = df_kor2.pivot(index=["기준일자","시_구분","행정동명",'행정동_코드','연령대'], columns='시간대', values='그외_근무인구').reset_index().rename({"낮" : "낮_그외_근무인구","밤" : "밤_그외_근무인구"},axis=1)
            pop1_제주_근무 = df_kor2.pivot(index=["기준일자","시_구분","행정동명",'행정동_코드','연령대'], columns='시간대', values='제주_근무인구').reset_index().rename({"낮" : "낮_제주_근무인구","밤" : "밤_제주_근무인구"},axis=1)
            pop1_그외_방문 = df_kor2.pivot(index=["기준일자","시_구분","행정동명",'행정동_코드','연령대'], columns='시간대', values='그외_방문인구').reset_index().rename({"낮" : "낮_그외_방문인구","밤" : "밤_그외_방문인구"},axis=1)
            pop1_제주_방문 = df_kor2.pivot(index=["기준일자","시_구분","행정동명",'행정동_코드','연령대'], columns='시간대', values='제주_방문인구').reset_index().rename({"낮" : "낮_제주_방문인구","밤" : "밤_제주_방문인구"},axis=1)

            df_kor3 = pd.concat([pop1_제주_거주, pop1_그외_근무.iloc[:,5:], pop1_제주_근무.iloc[:,5:], pop1_그외_방문.iloc[:,5:], pop1_제주_방문.iloc[:,5:]], axis=1)
            df_kor3['연령그룹'] = df_kor3['연령대'].apply(lambda x: '10대_이하' if x in [0,10] else ('20대' if x==20 else ('30대' if x==30 else ('40_50대' if x in [40,50] else ('60대_이상')))))

            df_kor4 = df_kor3.groupby(['기준일자','시_구분','행정동명','행정동_코드','연령그룹']).sum().reset_index().drop(['연령대'], axis=1)

            pivot1 = df_kor4.pivot(index=["기준일자","시_구분","행정동명",'행정동_코드'], columns='연령그룹', values='낮_제주_거주인구').reset_index()
            pivot1.rename({"10대_이하" : "10대_이하_낮_제주_거주인구","20대" : "20대_낮_제주_거주인구","30대" : "30대_낮_제주_거주인구",
                        "40_50대" : "40_50대_낮_제주_거주인구", "60대_이상" : "60대_이상_낮_제주_거주인구"},axis=1, inplace=True)

            pivot2 = df_kor4.pivot(index=["기준일자","시_구분","행정동명",'행정동_코드'], columns='연령그룹', values='밤_제주_거주인구').reset_index()
            pivot2.rename({"10대_이하" : "10대_이하_밤_제주_거주인구","20대" : "20대_밤_제주_거주인구","30대" : "30대_밤_제주_거주인구",
                        "40_50대" : "40_50대_밤_제주_거주인구", "60대_이상" : "60대_이상_밤_제주_거주인구"},axis=1, inplace=True)

            pivot3 = df_kor4.pivot(index=["기준일자","시_구분","행정동명",'행정동_코드'], columns='연령그룹', values='낮_그외_근무인구').reset_index()
            pivot3.rename({"10대_이하" : "10대_이하_낮_그외_근무인구","20대" : "20대_낮_그외_근무인구","30대" : "30대_낮_그외_근무인구",
                        "40_50대" : "40_50대_낮_그외_근무인구", "60대_이상" : "60대_이상_낮_그외_근무인구"},axis=1, inplace=True)

            pivot4 = df_kor4.pivot(index=["기준일자","시_구분","행정동명",'행정동_코드'], columns='연령그룹', values='밤_그외_근무인구').reset_index()
            pivot4.rename({"10대_이하" : "10대_이하_밤_그외_근무인구","20대" : "20대_밤_그외_근무인구","30대" : "30대_밤_그외_근무인구",
                        "40_50대" : "40_50대_밤_그외_근무인구", "60대_이상" : "60대_이상_밤_그외_근무인구"},axis=1, inplace=True)

            pivot5 = df_kor4.pivot(index=["기준일자","시_구분","행정동명",'행정동_코드'], columns='연령그룹', values='낮_제주_근무인구').reset_index()
            pivot5.rename({"10대_이하" : "10대_이하_낮_제주_근무인구","20대" : "20대_낮_제주_근무인구","30대" : "30대_낮_제주_근무인구",
                        "40_50대" : "40_50대_낮_제주_근무인구", "60대_이상" : "60대_이상_낮_제주_근무인구"},axis=1, inplace=True)
            
            pivot10 = df_kor4.pivot(index=["기준일자","시_구분","행정동명",'행정동_코드'], columns='연령그룹', values='밤_제주_근무인구').reset_index()
            pivot10.rename({"10대_이하" : "10대_이하_밤_제주_근무인구","20대" : "20대_밤_제주_근무인구","30대" : "30대_밤_제주_근무인구",
                        "40_50대" : "40_50대_밤_제주_근무인구", "60대_이상" : "60대_이상_밤_제주_근무인구"},axis=1, inplace=True)
            
            pivot6 = df_kor4.pivot(index=["기준일자","시_구분","행정동명",'행정동_코드'], columns='연령그룹', values='낮_그외_방문인구').reset_index()
            pivot6.rename({"10대_이하" : "10대_이하_낮_그외_방문인구","20대" : "20대_낮_그외_방문인구","30대" : "30대_낮_그외_방문인구",
                        "40_50대" : "40_50대_낮_그외_방문인구", "60대_이상" : "60대_이상_낮_그외_방문인구"},axis=1, inplace=True)

            pivot7 = df_kor4.pivot(index=["기준일자","시_구분","행정동명",'행정동_코드'], columns='연령그룹', values='밤_그외_방문인구').reset_index()
            pivot7.rename({"10대_이하" : "10대_이하_밤_그외_방문인구","20대" : "20대_밤_그외_방문인구","30대" : "30대_밤_그외_방문인구",
                        "40_50대" : "40_50대_밤_그외_방문인구", "60대_이상" : "60대_이상_밤_그외_방문인구"},axis=1, inplace=True)

            pivot8 = df_kor4.pivot(index=["기준일자","시_구분","행정동명",'행정동_코드'], columns='연령그룹', values='낮_제주_방문인구').reset_index()
            pivot8.rename({"10대_이하" : "10대_이하_낮_제주_방문인구","20대" : "20대_낮_제주_방문인구","30대" : "30대_낮_제주_방문인구",
                        "40_50대" : "40_50대_낮_제주_방문인구", "60대_이상" : "60대_이상_낮_제주_방문인구"},axis=1, inplace=True)

            pivot9 = df_kor4.pivot(index=["기준일자","시_구분","행정동명",'행정동_코드'], columns='연령그룹', values='밤_제주_방문인구').reset_index()
            pivot9.rename({"10대_이하" : "10대_이하_밤_제주_방문인구","20대" : "20대_밤_제주_방문인구","30대" : "30대_밤_제주_방문인구",
                        "40_50대" : "40_50대_밤_제주_방문인구", "60대_이상" : "60대_이상_밤_제주_방문인구"},axis=1, inplace=True)

            df_kor5 = pd.concat([pivot1, pivot2.iloc[:,4:], pivot3.iloc[:,4:], pivot4.iloc[:,4:], pivot5.iloc[:,4:], 
                                 pivot10.iloc[:,4:], pivot6.iloc[:,4:], pivot7.iloc[:,4:],
                                 pivot8.iloc[:,4:], pivot9.iloc[:,4:]], axis=1)

            del df_kor, df_kor1, df_kor2, df_kor3, df_kor4

            return df_kor5 

    def get_df_long(self, raw = False):
            df_long = pd.read_csv(self.path + '02-2_장기체류 외국인 유동인구_LONG_TERM_FRGN.CSV', encoding='cp949')
            df_long.columns = ['기준일자','시간','외국인_국적코드','시_구분','행정동_코드',
                                '행정동명','거주인구','근무인구','방문인구','거주지역행정구역']
            df_long['기준일자'] = pd.to_datetime(df_long['기준일자'])
            df_long['행정동_코드'] = df_long['행정동_코드'].astype(str)
            df_long['시간대'] = df_long['시간'].apply(lambda x: '낮' if x in range(8,21) else '밤')
                
            if raw == True:
                df_long_raw = df_long.copy()
                del df_long
                return df_long_raw

            else:
                
                df_long = df_long.groupby(['기준일자','시간','시_구분','행정동명','거주지역행정구역','시간대']).sum().reset_index()
                
                장기거주_pop = df_long.pivot(index=['기준일자','시간','시_구분','행정동명',"시간대"],
                            columns='거주지역행정구역',values='거주인구').reset_index()
                장기근무_pop = df_long.pivot(index=['기준일자','시간','시_구분','행정동명',"시간대"],
                            columns='거주지역행정구역',values='근무인구').reset_index()
                장기방문_pop = df_long.pivot(index=['기준일자','시간','시_구분','행정동명',"시간대"],
                            columns='거주지역행정구역',values='방문인구').reset_index()
                
                장기거주_pop.rename({"그외" : "그외장기거주인구","제주" : "제주장기거주인구"},axis=1,inplace=True)
                장기근무_pop.rename({"그외" : "그외장기근무인구","제주" : "제주장기근무인구"},axis=1,inplace=True)
                장기방문_pop.rename({"그외" : "그외장기방문인구","제주" : "제주장기방문인구"},axis=1,inplace=True)
                
                장기pop= pd.merge(장기방문_pop, pd.merge(장기거주_pop,장기근무_pop, 
                                        left_on=['기준일자','시간','시_구분','행정동명',"시간대"], 
                                        right_on=['기준일자','시간','시_구분','행정동명',"시간대"]), 
                        left_on=['기준일자','시간','시_구분','행정동명',"시간대"], 
                        right_on=['기준일자','시간','시_구분','행정동명',"시간대"])
                
                del 장기거주_pop, 장기근무_pop, 장기방문_pop
                
                return 장기pop

    def get_df_term(self, raw = False):
            
            df_long = pd.read_csv(self.path + 'df_long.csv', encoding='UTF-8')
            df_short = pd.read_csv(self.path + '02-3_단기체류 외국인 유동인구_SHORT_TERM_FRGN.CSV', encoding='cp949')
            df_short.columns = ['기준일자','시간','외국인_국적코드','시_구분','행정동_코드','행정동명','방문인구']
            df_long['기준일자'] = pd.to_datetime(df_long['기준일자'])
            df_short['기준일자'] = pd.to_datetime(df_short['기준일자'])
            df_short['행정동_코드'] = df_short['행정동_코드'].astype(str)
            df_short['시간대'] = df_short['시간'].apply(lambda x: '낮' if x in range(8,21) else '밤')
            df_short.rename({'방문인구':'단기외국인방문인구'},axis=1, inplace=True)
                
            if raw == True:
                df_short_raw = df_short.copy()
                del df_short
                return df_short_raw
            
            else:
                
                df_short = df_short.groupby(['기준일자','시간','시_구분','행정동명','시간대']).sum().reset_index()
                df_merge = pd.merge(df_long, df_short, on=['기준일자','시간','시_구분','행정동명','시간대'],how='left')
                df_merge = df_merge.groupby(['기준일자','시_구분','행정동명','시간대']).sum().reset_index().drop('시간',axis=1)

                pivot1 = df_merge.pivot(index=['기준일자','시_구분','행정동명'],
                                        columns='시간대',values='그외장기방문인구').reset_index()
                pivot1.rename({"낮" : "낮_그외장기방문인구", "밤" : "밤_그외장기방문인구",},axis=1, inplace=True)

                pivot2 = df_merge.pivot(index=['기준일자','시_구분','행정동명'],
                                        columns='시간대',values='제주장기방문인구').reset_index()
                pivot2.rename({"낮" : "낮_제주장기방문인구", "밤" : "밤_제주장기방문인구",},axis=1, inplace=True)


                pivot3 = df_merge.pivot(index=['기준일자','시_구분','행정동명'],
                                        columns='시간대',values='제주장기거주인구').reset_index()
                pivot3.rename({"낮" : "낮_제주장기거주인구", "밤" : "밤_제주장기거주인구",},axis=1, inplace=True)


                pivot4 = df_merge.pivot(index=['기준일자','시_구분','행정동명'],
                                        columns='시간대',values='그외장기근무인구').reset_index()
                pivot4.rename({"낮" : "낮_그외장기근무인구", "밤" : "밤_그외장기근무인구",},axis=1, inplace=True)


                pivot5 = df_merge.pivot(index=['기준일자','시_구분','행정동명'],
                                        columns='시간대',values='제주장기근무인구').reset_index()
                pivot5.rename({"낮" : "낮_제주장기근무인구", "밤" : "밤_제주장기근무인구",},axis=1, inplace=True)

                pivot6 = df_merge.pivot(index=['기준일자','시_구분','행정동명'],
                                        columns='시간대',values='단기외국인방문인구').reset_index()
                pivot6.rename({"새벽" : "새벽_단기외국인방문인구","아침" : "아침_단기외국인방문인구",
                            "낮" : "낮_단기외국인방문인구", "밤" : "밤_단기외국인방문인구",},axis=1, inplace=True)
                
                df_term = pd.concat([pivot1, pivot2.iloc[:,3:], pivot3.iloc[:,3:], 
                                pivot4.iloc[:,3:], pivot5.iloc[:,3:], pivot6.iloc[:,3:]], axis=1)
                
                del df_long
                return df_term

In [366]:
class MergeAll():

    def __init__(self, path):
        self.path = path

    def merge_first(self, df_fw, df_kor, df_card, df_term):

        self.df_fw = df_fw
        self.df_kor = df_kor
        self.df_card = df_card
        self.df_term = df_term

        """

        merge_first
        제공 데이터 합치기 + 기본 파생변수 생성

        """
        merge1 = pd.merge(self.df_fw, self.df_card, 
                          on=[col for col in self.df_fw.columns if col in self.df_card.columns], how='left')
        merge2 = pd.merge(merge1, self.df_kor, on=[col for col in merge1 if col in self.df_kor.columns], how='left')
        merge3 = pd.merge(merge2, self.df_term, on=[col for col in merge2 if col in self.df_term.columns], how='left')


        df = merge3.copy()

        ## 날짜관련 변수 생성
        df['year'] = df['기준일자'].dt.year
        df['month'] = df['기준일자'].dt.month
        df['day'] = df['기준일자'].dt.day

        df['year_month'] = df['기준일자'].apply(lambda x: dt.date(x.year, x.month, day=1))
        df['dayofyear'] = df['기준일자'].dt.dayofyear
        df['dayofweek_num'] = df['기준일자'].dt.dayofweek
        df['dayofweek_name'] = df['기준일자'].dt.day_name()
        df['weeknumber'] = df['기준일자'].dt.strftime('%V')
        #df_merge['daynumber'] = df_merge['기준일자'].dt.strftime('%j')
        df['season'] = df['기준일자'].apply(lambda x: 'spring' if x.month in [3,4,5] 
                    else('summer' if x.month in [6,7,8] 
                        else('winter' if x.month in [12,1,2] 
                                else('autumn'))))
        df['is_weekend'] = df['dayofweek_name'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)
        df['month_cos'] = np.cos(2*np.pi*df['month']/6)
        df['time_idx'] = df['기준일자'].apply(lambda x: x.toordinal() - 736694)


        ## 클러스터 변수
        clust1 = pd.read_csv(self.path + '거주방문인구_클러스터링.csv')
        clust1.drop(['행정동_코드'], axis=1, inplace=True)
        df = pd.merge(df, clust1, on='행정동명', how='left')

        clust2 = pd.read_csv(self.path + 'kmeans_clus_df.csv')
        df = pd.merge(df, clust2, on='행정동명', how='left')

        # 알수없음 삭제
        df = df[df['행정동명']!='알수없음'].copy()

        return df

    def merge_second(self, data):
        self.data = data 

        """

        merge_second
        외부데이터 합치기

        """

        ## 코로나 데이터
        covid_total = pd.read_csv(self.path + 'covid_total.csv')
        covid_total['기준일자'] = pd.to_datetime(covid_total['기준일자'])
        covid_total.drop_duplicates(inplace=True)
        df = pd.merge(self.data, covid_total, on=['기준일자'], how='left')
        #df = df.fillna(0)

        ## 날씨 데이터
        weather = pd.read_csv(self.path + 'OBS_ASOS_DD_20210913044124.csv', encoding='cp949')
        weather.drop(['지점'], axis=1, inplace=True)
        weather['일시'] = pd.to_datetime(weather['일시'])
        weather.fillna(0, inplace=True)
        weather.rename({'일시':'기준일자','지점명':'시_구분'}, axis=1, inplace=True)
        weather.loc[weather['시_구분']=='제주','시_구분'] = '제주시'
        weather.loc[weather['시_구분']=='서귀포','시_구분'] = '서귀포시'

        weather = weather.fillna(0)
        df = pd.merge(df, weather, on=['기준일자','시_구분'], how='left')

        ## 태풍 데이터 
        hurri = pd.read_csv(self.path + 'FCT_태풍목록_20210823195839.csv', encoding='cp949')
        hurri = hurri[hurri['영향도']!='없음']
        hurri['발생'] = pd.to_datetime(hurri['발생'], format='%Y.%m.%d %H:%M').dt.date
        hurri['소멸'] = pd.to_datetime(hurri['소멸'], format='%Y.%m.%d %H:%M').dt.date
        hurri.reset_index(drop=True, inplace=True)

        df['기준일자'] = df['기준일자'].dt.date

        for i in range(hurri.shape[0]):
            booll = df['기준일자'].apply(lambda x: hurri.loc[i,'발생'] <= x <= hurri.loc[i,'소멸'])
            df.loc[booll,'태풍'] = hurri.loc[i,'영향도']
        df['태풍'] = df['태풍'].fillna('없음')
        df['태풍'] = df['태풍'].apply(lambda x: 2 if x=='직접영향' else (1 if x=='상륙' else 0))
        df['기준일자'] = pd.to_datetime(df['기준일자'])

        ## 검색어 데이터
        search = pd.read_csv(self.path + '검색어데이터.csv')
        search['기준일자'] = pd.to_datetime(search['기준일자'])
        df = pd.merge(df, search, on='기준일자', how='left')

        ## 거주인구 데이터
        resid = pd.read_csv(self.path + 'resid_pivot.csv')
        resid.rename({'읍면동명':'행정동명', '읍면동_코드':'행정동_코드'}, axis=1, inplace=True)
        resid['행정동_코드'] = resid['행정동_코드'].astype(str)
        resid['year_month'] = resid.apply(lambda x: datetime(year=x.기준년, month=x.기준월, day=1), axis=1)
        df['year_month'] = pd.to_datetime(df['year_month'])
        df['행정동_코드'] = df['행정동_코드'].astype(str)

        df = pd.merge(df, resid, on=['year_month','시_구분','행정동명','행정동_코드'], how='left')
        df.drop(['기준년','기준월'], axis=1, inplace=True)


        ## 특일정보 데이터
        df_holidays = pd.read_csv(self.path + 'df_holidays.csv')
        df_holidays.drop(['dateKind'], axis=1, inplace=True)
        df_holidays.rename({'locdate':'기준일자'}, axis=1, inplace=True)
        df_holidays['기준일자'] = pd.to_datetime(df_holidays['기준일자'])

        df_holidays['isHoliday'] = df_holidays['isHoliday'].apply(lambda x: 1 if x=='Y' else 0)
        df_holidays.loc[df_holidays['dateName'].isin(['초복','중복','말복']),'isHoliday'] = 1
        df_holidays = df_holidays[df_holidays['isHoliday']==1]
        df_holidays.drop_duplicates(['기준일자'], inplace=True)
        df = pd.merge(df, df_holidays, on='기준일자', how='left')

        ## 아파트 데이터
        hjd_mapping = pd.read_csv(self.path + '법정동_행정동_매핑.csv')

        hjd_mapping.rename({'lgdng_cd':'법정동코드',
                            'ctpv_nm':'시도명',
                            'ctgg_nm':'시군구명',
                            'adstrd_nm':'읍면동명2',
                            'adstrd_en_nm':'행정동영문명',
                            'lgdng_nm':'읍면동명1',
                            'adstrd_cd':'행정동코드',
                            'admn_inst_cd':'행정기관코드'}, axis=1, inplace=True)

        hjd_mapping.drop(['행정동영문명','읍면동명1','행정동코드'], axis=1, inplace=True)
        hjd_mapping.rename({'읍면동명2':'행정동명'}, axis=1, inplace=True)
        hjd_mapping['법정동코드'] = hjd_mapping['법정동코드'].astype(str)
        hjd_mapping['행정기관코드'] = hjd_mapping['행정기관코드'].astype(str)

        hjd_mapping_jeju = hjd_mapping[hjd_mapping['시도명']=='제주특별자치도']
        hjd_mapping_jeju.drop(['시도명'], axis=1, inplace=True)


        df_apt_loc = pd.read_csv(self.path + 'apt_loc.csv')
        df_apt_loc = df_apt_loc[df_apt_loc['행법정동코드'].isna()==False]
        df_apt_loc['행법정동코드'] = df_apt_loc['행법정동코드'].astype(int).astype(str)
        #df_apt_loc.drop(['X좌표','Y좌표','등록일시','상세주소'], axis=1, inplace=True)
        #df_apt_loc.drop(['X좌표','Y좌표','상세주소'], axis=1, inplace=True) # 등록일시를 고려해야할지 고민...
        df_apt_loc = df_apt_loc[df_apt_loc['행법정동코드'].apply(lambda x: x[0]=='5')] # 제주 아닌 것들 제외

        temp = pd.merge(df_apt_loc, hjd_mapping_jeju, left_on='행법정동코드', right_on='법정동코드', how='left')
        temp['행정동_코드'] = temp['행정기관코드']
        temp.loc[temp['법정동코드'].isna(),'행정동_코드'] = temp.loc[temp['법정동코드'].isna(),'행법정동코드']
        temp = temp[['아파트코드', '아파트명', '기본주소', '세대수','행정동_코드']]

        temp = pd.merge(temp, hjd_mapping_jeju.drop(['법정동코드'],axis=1).drop_duplicates(), left_on='행정동_코드', right_on='행정기관코드', how='left')
        temp['행정동_코드'] = temp['행정동_코드'].apply(lambda x: x[:-2])

        #dup_apt = temp.loc[temp.duplicated(['아파트코드']),'아파트코드']

        dup_apt_new = pd.read_csv(self.path + 'dup_apt.csv', encoding='cp949')
        dup_apt_new = dup_apt_new[['아파트코드','아파트명','행정동명_new']]
        dup_apt_new = dup_apt_new[dup_apt_new['행정동명_new'].isna()==False] # 중복됐던 아파트의 정확한 행정동명

        temp2 = pd.merge(temp, dup_apt_new, on=['아파트코드','아파트명'], how='left')
        temp2.loc[temp2['행정동명_new'].isna(),'행정동명_new'] = temp2.loc[temp2['행정동명_new'].isna(),'행정동명']
        temp2 = temp2.drop(['행정동명'], axis=1).rename({'행정동명_new':'행정동명'}, axis=1)
        temp2.drop_duplicates(['아파트코드'], inplace=True)

        df_apt_count = temp2.groupby(['행정동명']).sum().reset_index()
        df = pd.merge(df, df_apt_count, on=['행정동명'], how='left')

        ## 신선식품지수, 생활물가지수
        food_index = pd.read_csv(self.path + '신선식품지수.csv', encoding='cp949')
        price_index = pd.read_csv(self.path + '생활물가지수.csv', encoding='cp949')

        food_index['시점'] = pd.to_datetime(food_index['시점'])
        food_index.rename({'시점':'year_month','제주특별자치도':'신선식품지수'}, axis=1, inplace=True)

        price_index['시점'] = pd.to_datetime(price_index['시점'])
        price_index.rename({'시점':'year_month','제주특별자치도':'생활물가지수'}, axis=1, inplace=True)

        df = pd.merge(df, food_index, on='year_month')
        df = pd.merge(df, price_index, on='year_month')

        # 학교 정보 
        df_hjd_infra = pd.read_csv(self.path + '행정동인프라.csv')
        df = pd.merge(df, df_hjd_infra, on=['행정동명'], how='left')


        # 카드 데이터 파생변수        
        df['외식_결제건수'] = df[['아시아음식_결제건수', '양식_결제건수', '주점및주류판매_결제건수', '패스트푸드_결제건수', '한식_결제건수']].sum(axis=1)
        df['외식_결제금액'] = df[['아시아음식_결제금액', '양식_결제금액', '주점및주류판매_결제금액', '패스트푸드_결제금액', '한식_결제금액']].sum(axis=1)

        df['1회당_외식_결제금액'] = df['외식_결제금액'] / df['외식_결제건수']
        df['1회당_마트/슈퍼마켓_결제금액'] = df['마트/슈퍼마켓_결제금액'] / df['마트/슈퍼마켓_결제건수']
        df['1회당_배달_결제금액'] = df['배달_결제금액'] / df['배달_결제건수']
        df['1회당_농축수산물_결제금액'] = df['농축수산물_결제금액'] / df['배달_결제건수']

        ############## 읍면동별 세대수/면적 데이터  #####################################
        df_house = pd.read_csv(self.path + "읍면동별_세대수_면적.csv",encoding='cp949')
        df_house = df_house[["시_구분","행정동명","한국인세대수","면적"]][df_house.year == 2020]
        df = pd.merge(df, df_house, on=['시_구분','행정동명'], how='left')


        ############## 세대당 카드결제 파생변수  #####################################
        group1 = df.groupby(['year','행정동명','한국인세대수']).sum().reset_index()[['year','행정동명','한국인세대수','배달_결제건수']]
        group1['세대당_배달_결제건수'] = group1['배달_결제건수'] / group1['한국인세대수']
        df = pd.merge(df,group1[['year','행정동명','세대당_배달_결제건수']], on=['year','행정동명'], how='left')

        group2 = df.groupby(['year','행정동명','한국인세대수']).sum().reset_index()[['year','행정동명','한국인세대수','마트/슈퍼마켓_결제금액']]
        group2['세대당_마트/슈퍼마켓_결제금액'] = group2['마트/슈퍼마켓_결제금액'] / group2['한국인세대수']
        df = pd.merge(df,group2[['year','행정동명','세대당_마트/슈퍼마켓_결제금액']], on=['year','행정동명'], how='left')

        group3 = df.groupby(['year','행정동명','한국인세대수']).sum().reset_index()[['year','행정동명','한국인세대수','외식_결제금액']]
        group3['세대당_외식_결제금액'] = group3['외식_결제금액'] / group3['한국인세대수']
        df = pd.merge(df,group3[['year','행정동명','세대당_외식_결제금액']], on=['year','행정동명'], how='left')

        group4 = df.groupby(['year','행정동명','한국인세대수']).sum().reset_index()[['year','행정동명','한국인세대수','농축수산물_결제금액']]
        group4['세대당_농축수산물_결제금액'] = group4['농축수산물_결제금액'] / group4['한국인세대수']
        df = pd.merge(df,group4[['year','행정동명','세대당_농축수산물_결제금액']], on=['year','행정동명'], how='left')

        del group1, group2, group3, group4

        ############## 스벅 대기업 개수 데이터  #####################################
        df_company = pd.read_csv(self.path + "스벅_대기업개수.csv",encoding='cp949')
        df = pd.merge(df, df_company, on=['시_구분', '행정동명'], how='left')


        ##############  읍면동 변수 #####################################

        df.loc[df['행정동명'].str.contains('동'),'읍면동'] = '동부'
        df.loc[df['행정동명'].str.contains('면'),'읍면동'] = '면부'
        df.loc[df['행정동명'].str.contains('읍'),'읍면동'] = '읍부'

        ##############  주가 데이터 #####################################

        trade_lotte = pd.read_csv(self.path + "롯데관광개발.csv")
        trade_hana = pd.read_csv(self.path + "하나투어.csv")
        trade_jejuair = pd.read_csv(self.path + "제주항공.csv")

        trade_lotte = trade_lotte.rename({'Date':'기준일자'}, axis=1)
        trade_hana = trade_hana.rename({'Date':'기준일자'}, axis=1)
        trade_jejuair = trade_jejuair.rename({'Date':'기준일자'}, axis=1)

        trade_lotte['기준일자'] = pd.to_datetime(trade_lotte['기준일자'])
        trade_hana['기준일자'] = pd.to_datetime(trade_hana['기준일자'])
        trade_jejuair['기준일자'] = pd.to_datetime(trade_jejuair['기준일자'])


        df = pd.merge(df, trade_lotte[['기준일자','Open']].rename({'Open':'롯데관광주가'}, axis=1), on=['기준일자'], how='left')
        df = pd.merge(df, trade_hana[['기준일자','Open']].rename({'Open':'하나투어주가'}, axis=1), on=['기준일자'], how='left')
        df = pd.merge(df, trade_jejuair[['기준일자','Open']].rename({'Open':'제주항공주가'}, axis=1), on=['기준일자'], how='left')
        df[['롯데관광주가','하나투어주가','제주항공주가']] = df[['롯데관광주가','하나투어주가','제주항공주가']].fillna(method="bfill")

        ############## 세대구성 데이터 #####################################

        df_house2 = pd.read_csv(self.path + "세대구성.csv", encoding='cp949', na_values='X')
        df_house2.fillna(0, inplace=True)
        df_house2['세대1'] = 0

        for col in df_house2.columns:
            if col.startswith('1세대가구-'):
                df_house2['세대1'] += df_house2[col]

        df_house2['세대2'] = 0
        for col in df_house2.columns:
            if col.startswith('2세대가구-'):
                df_house2['세대2'] += df_house2[col]

        df_house2['세대3'] = 0
        for col in df_house2.columns:
            if col.startswith('3세대가구-'):
                df_house2['세대3'] += df_house2[col]

        df_house2.rename({'4세대이상 가구 (가구)':'세대4이상', '1인가구 (가구)':'1인가구'}, axis=1, inplace=True)
        df_house2['1인가구비율'] = df_house2['1인가구']/df_house2['일반가구 (가구)']
        df_house2 = df_house2[['시점','행정구역별(시군구)','세대1','세대2','세대3','세대4이상','1인가구','1인가구비율']].rename({'시점':'year','행정구역별(시군구)':'읍면동'}, axis=1)
        #df_house2.year = df_house2.year.astype("str")
        df = pd.merge(df, df_house2, on=['year','읍면동'], how='left')


        ############## 세대구성 데이터 #####################################

        df_load = pd.read_csv(self.path + "화물량.csv")
        df_load['year_month'] = pd.to_datetime(df_load['year_month'], format='%Y%m')
        df = pd.merge(df, df_load, on=['year_month'], how='left')


        ############## 등교여부 변수 #####################################

        df.loc[(df.기준일자<='2018-03-02'), '초등학교등교여부'] = 0
        df.loc[(df.기준일자>='2018-07-24') & (df.기준일자<='2018-08-23'), '초등학교등교여부'] = 0
        df.loc[(df.기준일자>='2019-01-04') & (df.기준일자<='2019-03-04'), '초등학교등교여부'] = 0
        df.loc[(df.기준일자>='2019-07-24') & (df.기준일자<='2019-08-26'), '초등학교등교여부'] = 0
        df.loc[(df.기준일자>='2019-12-27') & (df.기준일자<='2020-05-20'), '초등학교등교여부'] = 0
        df.loc[(df.기준일자>='2020-08-14') & (df.기준일자<='2020-09-01'), '초등학교등교여부'] = 0
        df.loc[(df.기준일자>='2020-12-31') & (df.기준일자<='2021-03-02'), '초등학교등교여부'] = 0
        df.loc[(df.기준일자>='2021-07-23') & (df.기준일자<='2021-08-23'), '초등학교등교여부'] = 0
        df.초등학교등교여부 = df.초등학교등교여부.fillna(1)

        df.loc[df['기준일자'] <= "2018-03-02",'중학교등교여부'] = 0
        df.loc[(df['기준일자']>="2018-07-20") & (df['기준일자']<="2018-08-16"),'중학교등교여부'] = 0
        df.loc[(df['기준일자']>="2018-12-28") & (df['기준일자']<="2019-03-04"),'중학교등교여부'] = 0
        df.loc[(df['기준일자']>="2019-07-19") & (df['기준일자']<="2019-08-16"),'중학교등교여부'] = 0
        df.loc[(df['기준일자']>="2020-01-03") & (df['기준일자']<="2020-05-20"),'중학교등교여부'] = 0
        df.loc[(df['기준일자']>="2020-08-18") & (df['기준일자']<="2020-09-01"),'중학교등교여부'] = 0
        df.loc[(df['기준일자']>="2021-01-07") & (df['기준일자']<="2021-03-02"),'중학교등교여부'] = 0
        df.loc[(df['기준일자']>="2021-07-14") & (df['기준일자']<="2020-08-17"),'중학교등교여부'] = 0
        df["중학교등교여부"] = df["중학교등교여부"].fillna(1)

        df.loc[(df.기준일자>='2018-01-08') & (df.기준일자<='2018-03-02'), '고등학교등교여부'] = 0
        df.loc[(df.기준일자>='2018-07-20') & (df.기준일자<='2018-08-16'), '고등학교등교여부'] = 0
        df.loc[(df.기준일자>='2018-12-28') & (df.기준일자<='2019-03-02'), '고등학교등교여부'] = 0
        df.loc[(df.기준일자>='2019-07-19') & (df.기준일자<='2019-08-12'), '고등학교등교여부'] = 0
        df.loc[(df.기준일자>='2019-12-27') & (df.기준일자<='2020-05-20'), '고등학교등교여부'] = 0
        df.loc[(df.기준일자>='2020-08-13') & (df.기준일자<='2020-08-31'), '고등학교등교여부'] = 0
        df.loc[(df.기준일자>='2021-01-07') & (df.기준일자<='2021-03-02'), '고등학교등교여부'] = 0
        df.loc[(df.기준일자>='2021-07-16') & (df.기준일자<='2021-08-17'), '고등학교등교여부'] = 0
        df.고등학교등교여부 = df.고등학교등교여부.fillna(1)


        ############## 기타 변수 #####################################
        
        # 연말연시
        df.loc[(df['기준일자']<"2018-01-10"),'period'] = '연말연시'
        df.loc[("2018-12-20"<df['기준일자']) & (df['기준일자']<"2019-01-10"),'period'] = '연말연시'
        df.loc[("2019-12-20"<df['기준일자']) & (df['기준일자']<"2020-01-10"),'period'] = '연말연시'
        df.loc[("2020-12-20"<df['기준일자']) & (df['기준일자']<"2021-01-10"),'period'] = '연말연시'

        # 올림픽기간 
        df.loc[("2018-02-09"<=df['기준일자']) & (df['기준일자']<"2018-02-25"),'period'] ='sports'
        df.loc[("2021-07-23"<=df['기준일자']) & (df['기준일자']<"2021-08-08"),'period'] = 'sports'
        df.loc[("2018-06-18"<=df['기준일자']) & (df['기준일자']<"2018-07-15"),'period'] = 'sports'

        # 나머지 기간은 NA 대신 일반으로
        df['period'] = df['period'].fillna('일반')
        df.loc[df['month'].isin([7]),'period'] = '장마'

        ############## 캠핑 데이터 #####################################

        df1=pd.read_csv(self.path + '6500000_제주특별자치도_03_11_07_P_일반야영장업.csv',encoding='cp949')
        df2=pd.read_csv(self.path + '6500000_제주특별자치도_03_11_05_P_자동차야영장업.csv',encoding='cp949')
        df_camp = pd.concat([df1,df2])

        df_camp = df_camp[['인허가일자','폐업일자','관리번호','소재지전체주소']]
        df_camp['법정동명']=df_camp["소재지전체주소"].str.split(" ").str[2]

        dff = pd.read_csv(self.path + "법정동_행정동_매핑.csv")
        df_camp2 = pd.merge(df_camp, dff[['adstrd_nm', 'lgdng_nm']], left_on=['법정동명'], right_on=["lgdng_nm"],
                            how='left').rename({'adstrd_nm':'행정동명'}, axis=1)

        df_camp2.폐업일자 = df_camp2.폐업일자.fillna('20210901')
        df_camp2.폐업일자 = pd.to_datetime(df_camp2.폐업일자, format='%Y%m%d')
        df_camp2.인허가일자 = pd.to_datetime(df_camp2.인허가일자, format='%Y%m%d')

        df['캠핑장'] = 0
        df.기준일자 = pd.to_datetime(df.기준일자)
        for i in range(len(df_camp2)):
            ind = df[(df.행정동명==df_camp2.iloc[i]['행정동명']) & (df.기준일자 >= df_camp2.iloc[i]['인허가일자']) & (df.기준일자 < df_camp2.iloc[i]['폐업일자'])].index
            for ind1 in ind:
                df.loc[ind1,'캠핑장'] += 1

        ## 인구밀도
        df['density'] = df['총거주인구'] / df['면적']
        
        df['dateName'].fillna('일반', inplace=True)
        df['isHoliday'].fillna(0, inplace=True)


        df[['전국_확진자수', '전국_격리해제수',
        '전국_사망자수', '전국_전일대비_증감수', '제주_확진자수', '제주_격리해제수', '제주_사망자수',
        '제주_전일대비_증감수', '전국_격리중_환자수', '제주_격리중_환자수', '전국_해외유입수', '제주_해외유입수']] = \
        df[['전국_확진자수', '전국_격리해제수',
        '전국_사망자수', '전국_전일대비_증감수', '제주_확진자수', '제주_격리해제수', '제주_사망자수',
        '제주_전일대비_증감수', '전국_격리중_환자수', '제주_격리중_환자수', '전국_해외유입수', '제주_해외유입수']].fillna(0)
        
        ## 클러스터 데이터 추가
        kclus = pd.read_csv(self.path + 'kclus.csv')
        df = pd.merge(df, kclus, on='행정동명', how='left')
        df['year_month'] = pd.to_datetime(df['year_month'])
        
        kclus_month = pd.read_csv(self.path + 'df_kclus_month.csv')
        kclus_month['year_month'] = pd.to_datetime(kclus_month['year_month'])
        df = pd.merge(df, kclus_month, on=['year_month'], how='left')
        df['year_month'] = pd.to_datetime(df['year_month'])
        
        
        
        ## 카페 데이터
        df_cafe_train = pd.read_csv(self.path + "df_cafe_train.csv")
        df_cafe_train['기준일자'] = pd.to_datetime(df_cafe_train['기준일자'])
        df = pd.merge(df, df_cafe_train, on=['기준일자', '행정동명'], how='left')
        
        df['density'] = df['총거주인구'] / df['면적']
        df['평균_가구원수'] = df['총거주인구'] / df['한국인세대수']

        return df

In [367]:
## lag변수 만드는 함수
def create_lags(data, cols, lags, rolling_mean=False):
    
    ## lag
    if rolling_mean == False:
        for col in cols:
            for lag in lags:
                data[f'{col}_lag{lag}'] = data[['기준일자','행정동명',col]].groupby("행정동명")[col].shift(lag)
                data[f'{col}_lag{lag}'] = data[f'{col}_lag{lag}'].bfill()

        ## rolling mean of lag
    if rolling_mean == True:
        windows = [7,14]
        for col in cols:
            for lag in lags:
                data[f'{col}_lag{lag}'] = data[['기준일자','행정동명',col]].groupby("행정동명")[col].shift(lag)
                data[f'{col}_lag{lag}'] = data[f'{col}_lag{lag}'].bfill()
                
            lag_cols = [f'{col}_lag{lag}' for lag in lags]
            for window in windows:
                for lag, lag_col in zip(lags, lag_cols):
                    data[f'{col}_rmean{lag}_{window}'] = data[['기준일자','행정동명',lag_col]].groupby("행정동명")[lag_col].transform(lambda x : x.rolling(window).mean())
                    data[f'{col}_rmean{lag}_{window}'] = data[f'{col}_rmean{lag}_{window}'].bfill()

In [368]:
path = './final_data/' 
df_fw = GetData(path).get_df_fw() # 배출량 데이터 가져오기 (raw=False)
df_kor = GetData(path).get_df_kor() # 내국인 유동인구 데이터 가져오기 (raw=False)
df_card = GetData(path).get_df_card() # 카드 데이터 가져오기 
df_term = GetData(path).get_df_term() # 외국인 유동인구 데이터 가져오기 

/var/folders/lp/33v6zfq17kl531_hmpb5rwph0000gn/T/ipykernel_29533/1036320737.py:2: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  df_fw = GetData(path).get_df_fw() # 배출량 데이터 가져오기 (raw=False)
/Users/jiwonpark/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [369]:
df1 = MergeAll(path).merge_first(df_fw, df_kor, df_card, df_term)
df2 = MergeAll(path).merge_second(df1)
df = df2.copy()

/Users/jiwonpark/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [370]:
create_lags(df, cols=['전체_코로나', '전체_숙박','전체_버스', 
                        '전체_맛집', '전체_렌트카', '전체_날씨', '전체_항공',
                        '전체_제주코로나'], lags = [1,3,5,7,14], rolling_mean=True)

create_lags(df, cols = ['전국_확진자수','전국_사망자수','전국_전일대비_증감수',
                        '제주_확진자수','제주_사망자수','제주_전일대비_증감수'],
            lags = [3,7,14,28], rolling_mean=True)

create_lags(df, cols=['평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '강수 계속시간(hr)', '10분 최다 강수량(mm)',
       '1시간 최다강수량(mm)', '일강수량(mm)', '최대 순간 풍속(m/s)', '최대 풍속(m/s)',
       '평균 풍속(m/s)', '평균 이슬점온도(°C)', '최소 상대습도(%)', '평균 상대습도(%)', '평균 증기압(hPa)',
       '평균 현지기압(hPa)', '최고 해면기압(hPa)', '최저 해면기압(hPa)', '평균 해면기압(hPa)',
       '합계 일조시간(hr)', '1시간 최다일사량(MJ/m2)', '합계 일사량(MJ/m2)', '평균 전운량(1/10)',
       '평균 중하층운량(1/10)', '합계 대형증발량(mm)', '합계 소형증발량(mm)', '안개 계속시간(hr)', '태풍'],
                lags = [1,3,5,7], rolling_mean=False)

In [371]:
df.to_csv('df_ver0914.csv', encoding='cp949', index=False)

In [372]:
df_columns = pd.DataFrame(df.columns)
df_columns.to_csv('df_columns.csv', index=False, encoding='cp949')

In [373]:
df.shape

(52357, 569)

In [374]:
df.head().T

,0,1,2,3,4
기준일자,2018-01-01 00:00:00,2018-01-02 00:00:00,2018-01-03 00:00:00,2018-01-04 00:00:00,2018-01-05 00:00:00
행정동명,건입동,건입동,건입동,건입동,건입동
배출량,1708250.0,1841150.0,1411450.0,1558700.0,1338350.0
시_구분,제주시,제주시,제주시,제주시,제주시
행정동_코드,50110590,50110590,50110590,50110590,50110590
배출건수,668.0,745.0,563.0,652.0,595.0
총_지불금액,51079.0,55047.0,42203.0,46606.0,39996.0
거점지역_개수,29.0,29.0,29.0,29.0,29.0
간식_결제건수,79.0,137.0,177.0,137.0,93.0
농축수산물_결제건수,34.0,44.0,49.0,74.0,49.0


# df_test

In [375]:
class CreateTest():
    def __init__(self, path):
        self.path = path
        
    def create_test(self, df):
        self.df = df
        
        date = pd.date_range(start='20210701', end='20210831')
        dt_list = date.strftime("%Y-%m-%d").tolist()*41
        
        ## 기준일자/행정동명
        region = ['건입동', '남원읍', '노형동', '대륜동', '대정읍', '대천동', '도두동', '동홍동', '봉개동',
               '삼도1동', '삼도2동', '삼양동', '서홍동', '성산읍', '송산동', '아라동', '안덕면', '애월읍',
               '연동', '영천동', '예래동', '오라동', '외도동', '용담1동', '용담2동', '이도1동', '이도2동',
               '이호동', '일도1동', '일도2동', '정방동', '중문동', '중앙동', '천지동', '표선면', '화북동',
               '효돈동', '한경면', '조천읍', '한림읍', '구좌읍']

        region_list =  list(chain.from_iterable((repeat(region, 62 ) for region in region)))
        df_test = pd.DataFrame({"기준일자":dt_list,"행정동명":region_list})
        
        ## 정적변수 추가
        df_region = self.df[['시_구분', '행정동_코드',"행정동명",'clust_방문거주', 'clust_방문거주2', 'kclus_배출', 'kclus_거주연령', 'kclus_시간유동',
               'kclus_방문유동', 'kclus_카드소비','상가',
               '숙박', '음식', '학문/교육', '식료품소매', '모텔/여관/여인숙', '민박/하숙', '유스호스텔', '캠프/별장/펜션',
               '호텔/콘도', '초등학교개수', '중학교개수', '고등학교개수', '학교개수']]
        df_test = pd.merge(df_test,df_region.drop_duplicates(),how="left",on="행정동명")
        
        del df_region
        
        ## 날짜관련 변수 생성 
        df_test['기준일자'] = pd.to_datetime(df_test['기준일자'])
        df_test['year'] = df_test['기준일자'].dt.year
        df_test['month'] = df_test['기준일자'].dt.month
        df_test['day'] = df_test['기준일자'].dt.day
        
        df_test['year_month'] = df_test['기준일자'].apply(lambda x: dt.datetime(x.year, x.month, day=1))
        df_test['dayofyear'] = df_test['기준일자'].dt.dayofyear
        df_test['dayofweek_num'] = df_test['기준일자'].dt.dayofweek
        df_test['dayofweek_name'] = df_test['기준일자'].dt.day_name()
        df_test['weeknumber'] = df_test['기준일자'].dt.strftime('%V')
        #df_merge['daynumber'] = df_merge['기준일자'].dt.strftime('%j')
        df_test['season'] = df_test['기준일자'].apply(lambda x: 'spring' if x.month in [3,4,5] 
                    else('summer' if x.month in [6,7,8] 
                        else('winter' if x.month in [12,1,2] 
                                else('autumn'))))
        df_test['is_weekend'] = df_test['dayofweek_name'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)
        df_test['month_cos'] = np.cos(2*np.pi*df['month']/6)
        df_test['year_month'] = pd.to_datetime(df_test['year_month'])
        df_test['time_idx'] = df_test['기준일자'].apply(lambda x: x.toordinal() - 736694)
        
        
        
        ##############  외부데이터  ######################################
        ###############################################################
        
        ############## 코로나 데이터 #####################################
        covid = pd.read_csv(self.path + "covid_test.csv")
        covid_jeju = covid[covid.지역 == "제주"].drop(["지역"],axis=1)

        covid_jeju['등록일자'] = pd.to_datetime(covid_jeju['등록일자'])
        covid['등록일자'] = pd.to_datetime(covid['등록일자'])

        covid_sum = covid.groupby('등록일자').sum().reset_index()
        covid_sum.columns = [f'전국_{col}' for col in covid_sum.columns]
        covid_sum.rename(columns = {"전국_등록일자":"기준일자"},inplace=True)

        covid_jeju.columns = [f'제주_{col}' for col in covid_jeju.columns]
        covid_jeju.rename(columns = {"제주_등록일자":"기준일자"},inplace=True)
        
        df_covid = pd.merge(covid_sum, covid_jeju, on='기준일자', how='inner')
        df_covid=df_covid.drop_duplicates()
        df_covid.drop(["제주_시도명","전국_지역발생수","제주_지역발생수"],axis=1,inplace=True) 
        #merge
        df_test = pd.merge(df_test, df_covid, on=['기준일자'], how='left')
        
        del covid, covid_jeju, covid_sum, df_covid

        
       ############## 날씨 데이터 #####################################
        weather = pd.read_csv(self.path + 'OBS_ASOS_DD_20210913044124.csv', encoding='cp949')

        weather.drop(['지점'], axis=1, inplace=True)
        weather['일시'] = pd.to_datetime(weather['일시'])
        #weather.columns = ['지점명', '일시', '평균기온', '최저기온', '최고기온', '강수 계속시간',
        #                    '일강수량', '최대 풍속', '평균 풍속', '안개 계속시간']
        weather.rename({'일시':'기준일자','지점명':'시_구분'}, axis=1, inplace=True)
        weather.loc[weather['시_구분']=='제주','시_구분'] = '제주시'
        weather.loc[weather['시_구분']=='서귀포','시_구분'] = '서귀포시'
        weather.fillna(0, inplace=True)
        df_test = pd.merge(df_test, weather, on=['기준일자','시_구분'], how='left')
        
        # 장마 (호우주의보) 
        df_test.loc[df_test['month'].isin([7]),'period'] = '장마'

        del weather
        print(df_test[df_test[['기준일자','행정동명']].duplicated()].shape)
        
        ############## 태풍 데이터 #####################################
        hurri = pd.read_csv(self.path + 'FCT_태풍목록_20210823195839.csv', encoding='cp949')
        hurri = hurri[hurri['영향도']!='없음']
        hurri['발생'] = pd.to_datetime(hurri['발생'], format='%Y.%m.%d %H:%M').dt.date
        hurri['소멸'] = pd.to_datetime(hurri['소멸'], format='%Y.%m.%d %H:%M').dt.date
        hurri.reset_index(drop=True, inplace=True)
        
        df_test['기준일자'] = df_test['기준일자'].dt.date

        for i in range(hurri.shape[0]):
            booll = df_test['기준일자'].apply(lambda x: hurri.loc[i,'발생'] <= x <= hurri.loc[i,'소멸'])
            df_test.loc[booll,'태풍'] = hurri.loc[i,'영향도']
        df_test['태풍'] = df_test['태풍'].fillna('없음')
        df_test['태풍'] = df_test['태풍'].apply(lambda x: 2 if x=='직접영향' else (1 if x=='상륙' else 0))
        df_test['기준일자'] = pd.to_datetime(df_test['기준일자'])
        
        del hurri
        
        
        ############## 검색어 데이터 #####################################
        search = pd.read_csv(self.path + '검색어데이터.csv')
        search['기준일자'] = pd.to_datetime(search['기준일자'])
        df_test = pd.merge(df_test, search, on='기준일자', how='left')
        
        del search
        
        
        ############## 거주인구 데이터 #####################################
        resid = pd.read_csv(self.path + "새로운_거주인구.csv" ,encoding='cp949')
        resid['행정동명'] = resid['행정동명'].str.strip()
        resid['year_month'] = pd.to_datetime(resid['year_month'])
        df_test['year_month'] = pd.to_datetime(df_test['year_month'])
        df_test = pd.merge(df_test, resid, on=["year_month","행정동명"], how= "left")
        
        del resid
        
       
        ############## 특일정보 데이터 #####################################
        holiday_2021 = pd.read_csv(self.path + "holiday_2021.csv",encoding="cp949")
        holiday_2021['기준일자'] = pd.to_datetime(holiday_2021['기준일자'])
        df_test = pd.merge(df_test, holiday_2021, on='기준일자', how='left')
        
        df_test['dateName'].fillna('일반', inplace=True)
        df_test['isHoliday'].fillna(0, inplace=True)

        
        del holiday_2021
        
        
        ############## 아파트 세대수 데이터 #####################################
        hjd_mapping = pd.read_csv(self.path + '법정동_행정동_매핑.csv')

        hjd_mapping.rename({'lgdng_cd':'법정동코드',
                            'ctpv_nm':'시도명',
                            'ctgg_nm':'시군구명',
                            'adstrd_nm':'읍면동명2',
                            'adstrd_en_nm':'행정동영문명',
                            'lgdng_nm':'읍면동명1',
                            'adstrd_cd':'행정동코드',
                            'admn_inst_cd':'행정기관코드'}, axis=1, inplace=True)

        hjd_mapping.drop(['행정동영문명','읍면동명1','행정동코드'], axis=1, inplace=True)
        hjd_mapping.rename({'읍면동명2':'행정동명'}, axis=1, inplace=True)
        hjd_mapping['법정동코드'] = hjd_mapping['법정동코드'].astype(str)
        hjd_mapping['행정기관코드'] = hjd_mapping['행정기관코드'].astype(str)

        hjd_mapping_jeju = hjd_mapping[hjd_mapping['시도명']=='제주특별자치도']
        hjd_mapping_jeju.drop(['시도명'], axis=1, inplace=True)
        


        df_apt_loc = pd.read_csv(self.path + 'apt_loc.csv')
        df_apt_loc = df_apt_loc[df_apt_loc['행법정동코드'].isna()==False]
        df_apt_loc['행법정동코드'] = df_apt_loc['행법정동코드'].astype(int).astype(str)
        #df_apt_loc.drop(['X좌표','Y좌표','등록일시','상세주소'], axis=1, inplace=True)
        #df_apt_loc.drop(['X좌표','Y좌표','상세주소'], axis=1, inplace=True) # 등록일시를 고려해야할지 고민...
        df_apt_loc = df_apt_loc[df_apt_loc['행법정동코드'].apply(lambda x: x[0]=='5')] # 제주 아닌 것들 제외

        temp = pd.merge(df_apt_loc, hjd_mapping_jeju, left_on='행법정동코드', right_on='법정동코드', how='left')
        temp['행정동_코드'] = temp['행정기관코드']
        temp.loc[temp['법정동코드'].isna(),'행정동_코드'] = temp.loc[temp['법정동코드'].isna(),'행법정동코드']
        temp = temp[['아파트코드', '아파트명', '기본주소', '세대수','행정동_코드']]

        temp = pd.merge(temp, hjd_mapping_jeju.drop(['법정동코드'],axis=1).drop_duplicates(), left_on='행정동_코드', right_on='행정기관코드', how='left')
        temp['행정동_코드'] = temp['행정동_코드'].apply(lambda x: x[:-2])

        dup_apt_new = pd.read_csv(self.path + 'dup_apt.csv', encoding='cp949')
        dup_apt_new = dup_apt_new[['아파트코드','아파트명','행정동명_new']]
        dup_apt_new = dup_apt_new[dup_apt_new['행정동명_new'].isna()==False] # 중복됐던 아파트의 정확한 행정동명

        temp2 = pd.merge(temp, dup_apt_new, on=['아파트코드','아파트명'], how='left')
        temp2.loc[temp2['행정동명_new'].isna(),'행정동명_new'] = temp2.loc[temp2['행정동명_new'].isna(),'행정동명']
        temp2 = temp2.drop(['행정동명'], axis=1).rename({'행정동명_new':'행정동명'}, axis=1)
        temp2.drop_duplicates(['아파트코드'], inplace=True)

        df_apt_count = temp2.groupby(['행정동명']).sum().reset_index()
        df_test = pd.merge(df_test, df_apt_count, on=['행정동명'], how='left')
        
        
        ############## 신선식품지수, 생활물가지수 데이터 #####################################
        food_index = pd.read_csv(self.path + '신선식품지수.csv', encoding='cp949')
        price_index = pd.read_csv(self.path + '생활물가지수.csv', encoding='cp949')

        food_index['시점'] = pd.to_datetime(food_index['시점'])
        food_index.rename({'시점':'year_month','제주특별자치도':'신선식품지수'}, axis=1, inplace=True)

        price_index['시점'] = pd.to_datetime(price_index['시점'])
        price_index.rename({'시점':'year_month','제주특별자치도':'생활물가지수'}, axis=1, inplace=True)
        
        df_test['year_month'] = pd.to_datetime(df_test['year_month'])
        df_test = pd.merge(df_test, food_index, on='year_month')
        df_test = pd.merge(df_test, price_index, on='year_month')
        
        
        ############## 읍면동별 세대수/면적 데이터  #####################################
        df_house = pd.read_csv(self.path + "읍면동별_세대수_면적.csv",encoding='cp949')
        df_house = df_house[["시_구분","행정동명","한국인세대수","면적"]][df_house.year == 2020]
        df_test = pd.merge(df_test, df_house, on=['시_구분','행정동명'], how='left')
        
        ############## 스벅 대기업 개수 데이터  #####################################
        df_company = pd.read_csv(self.path + "스벅_대기업개수.csv",encoding='cp949')
        df_test = pd.merge(df_test, df_company, on=['시_구분', '행정동명'], how='left')
        
        
        ##############  세대당 카드결제 파생변수 #####################################
        
        group1 = self.df.groupby(['year','행정동명','한국인세대수']).sum().reset_index()[['year','행정동명','한국인세대수','배달_결제건수']]
        group1['세대당_배달_결제건수'] = group1['배달_결제건수'] / group1['한국인세대수']
        df_test = pd.merge(df_test,group1[['year','행정동명','세대당_배달_결제건수']], on=['year','행정동명'], how='left')

        group2 = self.df.groupby(['year','행정동명','한국인세대수']).sum().reset_index()[['year','행정동명','한국인세대수','마트/슈퍼마켓_결제금액']]
        group2['세대당_마트/슈퍼마켓_결제금액'] = group2['마트/슈퍼마켓_결제금액'] / group2['한국인세대수']
        df_test = pd.merge(df_test,group2[['year','행정동명','세대당_마트/슈퍼마켓_결제금액']], on=['year','행정동명'], how='left')

        group3 = self.df.groupby(['year','행정동명','한국인세대수']).sum().reset_index()[['year','행정동명','한국인세대수','외식_결제금액']]
        group3['세대당_외식_결제금액'] = group3['외식_결제금액'] / group3['한국인세대수']
        df_test = pd.merge(df_test,group3[['year','행정동명','세대당_외식_결제금액']], on=['year','행정동명'], how='left')

        group4 = self.df.groupby(['year','행정동명','한국인세대수']).sum().reset_index()[['year','행정동명','한국인세대수','농축수산물_결제금액']]
        group4['세대당_농축수산물_결제금액'] = group4['농축수산물_결제금액'] / group4['한국인세대수']
        df_test = pd.merge(df_test,group4[['year','행정동명','세대당_농축수산물_결제금액']], on=['year','행정동명'], how='left')
        

        del group1, group2, group3, group4
        
        
        ##############  읍면동 변수 #####################################
        
        df_test.loc[df_test['행정동명'].str.contains('동'),'읍면동'] = '동부'
        df_test.loc[df_test['행정동명'].str.contains('면'),'읍면동'] = '면부'
        df_test.loc[df_test['행정동명'].str.contains('읍'),'읍면동'] = '읍부'
        
        ##############  주가 데이터 #####################################
        
        trade_lotte = pd.read_csv(self.path + "롯데관광개발.csv")
        trade_hana = pd.read_csv(self.path + "하나투어.csv")
        trade_jejuair = pd.read_csv(self.path + "제주항공.csv")
        
        trade_lotte = trade_lotte.rename({'Date':'기준일자'}, axis=1)
        trade_hana = trade_hana.rename({'Date':'기준일자'}, axis=1)
        trade_jejuair = trade_jejuair.rename({'Date':'기준일자'}, axis=1)

        trade_lotte['기준일자'] = pd.to_datetime(trade_lotte['기준일자'])
        trade_hana['기준일자'] = pd.to_datetime(trade_hana['기준일자'])
        trade_jejuair['기준일자'] = pd.to_datetime(trade_jejuair['기준일자'])
        

        df_test = pd.merge(df_test, trade_lotte[['기준일자','Open']].rename({'Open':'롯데관광주가'}, axis=1), on=['기준일자'], how='left')
        df_test = pd.merge(df_test, trade_hana[['기준일자','Open']].rename({'Open':'하나투어주가'}, axis=1), on=['기준일자'], how='left')
        df_test = pd.merge(df_test, trade_jejuair[['기준일자','Open']].rename({'Open':'제주항공주가'}, axis=1), on=['기준일자'], how='left')
        df_test[['롯데관광주가','하나투어주가','제주항공주가']] = df_test[['롯데관광주가','하나투어주가','제주항공주가']].fillna(method="bfill")
        
        ############## 세대구성 데이터 #####################################
        
        df_house2 = pd.read_csv(self.path + "세대구성.csv", encoding='cp949', na_values='X')
        df_house2.fillna(0, inplace=True)
        df_house2['세대1'] = 0

        for col in df_house2.columns:
            if col.startswith('1세대가구-'):
                df_house2['세대1'] += df_house2[col]

        df_house2['세대2'] = 0
        for col in df_house2.columns:
            if col.startswith('2세대가구-'):
                df_house2['세대2'] += df_house2[col]
                
        df_house2['세대3'] = 0
        for col in df_house2.columns:
            if col.startswith('3세대가구-'):
                df_house2['세대3'] += df_house2[col]

        df_house2.rename({'4세대이상 가구 (가구)':'세대4이상', '1인가구 (가구)':'1인가구'}, axis=1, inplace=True)
        df_house2['1인가구비율'] = df_house2['1인가구']/df_house2['일반가구 (가구)']
        df_house2 = df_house2[['시점','행정구역별(시군구)','세대1','세대2','세대3','세대4이상','1인가구','1인가구비율']].rename({'시점':'year','행정구역별(시군구)':'읍면동'}, axis=1)
        #df_house2.year = df_house2.year.astype("str")
        df_test = pd.merge(df_test, df_house2, on=['year','읍면동'], how='left')
        
        
        
        ############## 세대구성 데이터 #####################################
        
        df_load = pd.read_csv(self.path + "화물량.csv")
        df_load['year_month'] = pd.to_datetime(df_load['year_month'], format='%Y%m')
        df_test = pd.merge(df_test, df_load, on=['year_month'], how='left')
        
        
        ############## 등교여부 변수 #####################################
        
        df_test.loc[(df_test.기준일자<='2018-03-02'), '초등학교등교여부'] = 0
        df_test.loc[(df_test.기준일자>='2018-07-24') & (df_test.기준일자<='2018-08-23'), '초등학교등교여부'] = 0
        df_test.loc[(df_test.기준일자>='2019-01-04') & (df_test.기준일자<='2019-03-04'), '초등학교등교여부'] = 0
        df_test.loc[(df_test.기준일자>='2019-07-24') & (df_test.기준일자<='2019-08-26'), '초등학교등교여부'] = 0
        df_test.loc[(df_test.기준일자>='2019-12-27') & (df_test.기준일자<='2020-05-20'), '초등학교등교여부'] = 0
        df_test.loc[(df_test.기준일자>='2020-08-14') & (df_test.기준일자<='2020-09-01'), '초등학교등교여부'] = 0
        df_test.loc[(df_test.기준일자>='2020-12-31') & (df_test.기준일자<='2021-03-02'), '초등학교등교여부'] = 0
        df_test.loc[(df_test.기준일자>='2021-07-23') & (df_test.기준일자<='2021-08-23'), '초등학교등교여부'] = 0
        df_test.초등학교등교여부 = df_test.초등학교등교여부.fillna(1)

        df_test.loc[df_test['기준일자'] <= "2018-03-02",'중학교등교여부'] = 0
        df_test.loc[(df_test['기준일자']>="2018-07-20") & (df_test['기준일자']<="2018-08-16"),'중학교등교여부'] = 0
        df_test.loc[(df_test['기준일자']>="2018-12-28") & (df_test['기준일자']<="2019-03-04"),'중학교등교여부'] = 0
        df_test.loc[(df_test['기준일자']>="2019-07-19") & (df_test['기준일자']<="2019-08-16"),'중학교등교여부'] = 0
        df_test.loc[(df_test['기준일자']>="2020-01-03") & (df_test['기준일자']<="2020-05-20"),'중학교등교여부'] = 0
        df_test.loc[(df_test['기준일자']>="2020-08-18") & (df_test['기준일자']<="2020-09-01"),'중학교등교여부'] = 0
        df_test.loc[(df_test['기준일자']>="2021-01-07") & (df_test['기준일자']<="2021-03-02"),'중학교등교여부'] = 0
        df_test.loc[(df_test['기준일자']>="2021-07-14") & (df_test['기준일자']<="2020-08-17"),'중학교등교여부'] = 0
        df_test["중학교등교여부"] = df_test["중학교등교여부"].fillna(1)

        df_test.loc[(df_test.기준일자>='2018-01-08') & (df_test.기준일자<='2018-03-02'), '고등학교등교여부'] = 0
        df_test.loc[(df_test.기준일자>='2018-07-20') & (df_test.기준일자<='2018-08-16'), '고등학교등교여부'] = 0
        df_test.loc[(df_test.기준일자>='2018-12-28') & (df_test.기준일자<='2019-03-02'), '고등학교등교여부'] = 0
        df_test.loc[(df_test.기준일자>='2019-07-19') & (df_test.기준일자<='2019-08-12'), '고등학교등교여부'] = 0
        df_test.loc[(df_test.기준일자>='2019-12-27') & (df_test.기준일자<='2020-05-20'), '고등학교등교여부'] = 0
        df_test.loc[(df_test.기준일자>='2020-08-13') & (df_test.기준일자<='2020-08-31'), '고등학교등교여부'] = 0
        df_test.loc[(df_test.기준일자>='2021-01-07') & (df_test.기준일자<='2021-03-02'), '고등학교등교여부'] = 0
        df_test.loc[(df_test.기준일자>='2021-07-16') & (df_test.기준일자<='2021-08-17'), '고등학교등교여부'] = 0
        df_test.고등학교등교여부 = df_test.고등학교등교여부.fillna(1)
        
        
        
        ############## 기타 변수 #####################################
         
        # 연말연시
        df_test.loc[(df_test['기준일자']<"2018-01-10"),'period'] = '연말연시'
        df_test.loc[("2018-12-20"<df_test['기준일자']) & (df_test['기준일자']<"2019-01-10"),'period'] = '연말연시'
        df_test.loc[("2019-12-20"<df_test['기준일자']) & (df_test['기준일자']<"2020-01-10"),'period'] = '연말연시'
        df_test.loc[("2020-12-20"<df_test['기준일자']) & (df_test['기준일자']<"2021-01-10"),'period'] = '연말연시'
        

        # 올림픽기간 
        df_test.loc[("2018-02-09"<=df_test['기준일자']) & (df_test['기준일자']<"2018-02-25"),'period'] ='sports'
        df_test.loc[("2021-07-23"<=df_test['기준일자']) & (df_test['기준일자']<"2021-08-08"),'period'] = 'sports'
        df_test.loc[("2018-06-18"<=df_test['기준일자']) & (df_test['기준일자']<"2018-07-15"),'period'] = 'sports'
        

        # 나머지 기간은 NA 대신 일반으로
        df_test['period'] = df_test['period'].fillna('일반')
        
        
        ############## 캠핑 데이터 #####################################
        
        df1=pd.read_csv(self.path + '6500000_제주특별자치도_03_11_07_P_일반야영장업.csv',encoding='cp949')
        df2=pd.read_csv(self.path + '6500000_제주특별자치도_03_11_05_P_자동차야영장업.csv',encoding='cp949')
        df_camp = pd.concat([df1,df2])
        

        df_camp = df_camp[['인허가일자','폐업일자','관리번호','소재지전체주소']]
        df_camp['법정동명']=df_camp["소재지전체주소"].str.split(" ").str[2]
        

        dff = pd.read_csv(self.path + "법정동_행정동_매핑.csv")
        df_camp2 = pd.merge(df_camp, dff[['adstrd_nm', 'lgdng_nm']], left_on=['법정동명'], right_on=["lgdng_nm"],
                            how='left').rename({'adstrd_nm':'행정동명'}, axis=1)
        

        df_camp2.폐업일자 = df_camp2.폐업일자.fillna('20210901')
        df_camp2.폐업일자 = pd.to_datetime(df_camp2.폐업일자, format='%Y%m%d')
        df_camp2.인허가일자 = pd.to_datetime(df_camp2.인허가일자, format='%Y%m%d')
        

        df_test['캠핑장'] = 0
        df_test.기준일자 = pd.to_datetime(df_test.기준일자)
        for i in range(len(df_camp2)):
            ind = df_test[(df_test.행정동명==df_camp2.iloc[i]['행정동명']) & (df_test.기준일자 >= df_camp2.iloc[i]['인허가일자']) & (df_test.기준일자 < df_camp2.iloc[i]['폐업일자'])].index
            for ind1 in ind:
                df_test.loc[ind1,'캠핑장'] += 1
        
        ## 클러스터 데이터 추가
        kclus = pd.read_csv(self.path + 'kclus.csv')
        df_test = pd.merge(df_test, kclus, on='행정동명', how='left')
        df_test['year_month'] = pd.to_datetime(df_test['year_month'])
        
        kclus_month = pd.read_csv(self.path + 'df_kclus_month.csv')
        kclus_month['year_month'] = pd.to_datetime(kclus_month['year_month'])
        df_test = pd.merge(df_test, kclus_month, on=['year_month'], how='left')
        df_test['year_month'] = pd.to_datetime(df_test['year_month'])

       
        ## 카페 데이터
        df_cafe_test = pd.read_csv(self.path + "df_cafe_test.csv")
        df_cafe_test = df_cafe_test.drop_duplicates()
        df_cafe_test['기준일자'] = pd.to_datetime(df_cafe_test['기준일자'])
        df_test = pd.merge(df_test, df_cafe_test, on=['기준일자', '행정동명'], how='left')
        
        ## 인구밀도
        df_test['density'] = df_test['총거주인구'] / df_test['면적']
        df_test['평균_가구원수'] = df_test['총거주인구'] / df_test['한국인세대수']
        
        
        return df_test 

In [376]:
path = './final_data/'
df_test = CreateTest(path).create_test(df)

(0, 76)


/Users/jiwonpark/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [377]:
df["기준일자"] = pd.to_datetime(df['기준일자'], format='%Y-%m-%d')
df['time_idx'] = df['기준일자'].apply(lambda x: x.toordinal() - 736694)

df = df.drop(['행정동_코드','배출건수','총_지불금액'],axis=1)
df = df[df['기준일자'] >= '2021-05-01']

mean = df.groupby(['행정동명']).agg(np.mean).reset_index()[['행정동명','거점지역_개수']]
sd = df.groupby(['행정동명']).agg(np.std).reset_index()[['행정동명','거점지역_개수']]

mean.columns = ['행정동명', '거점지역개수_mean']
sd.columns = ['행정동명', '거점지역개수_sd']

df = pd.merge(df,mean, on=['행정동명'], how='left')
df = pd.merge(df,sd, on=['행정동명'], how='left')

df = df[['행정동명', '거점지역개수_mean', '거점지역개수_sd']].drop_duplicates().reset_index(drop = True)

df_test = pd.merge(df_test, df, on=['행정동명'], how='left')

df_test["기준일자"] = pd.to_datetime(df_test['기준일자'], format='%Y-%m-%d')
df_test['time_idx'] = df_test['기준일자'].apply(lambda x: x.toordinal() - 736694)

df_test.rename({'거점지역개수_mean':'거점지역_개수'}, axis=1, inplace=True)

In [378]:
create_lags(df_test, cols=['전체_코로나', '전체_숙박','전체_버스', 
                        '전체_맛집', '전체_렌트카', '전체_날씨', '전체_항공',
                        '전체_제주코로나'], lags = [1,3,5,7,14], rolling_mean=True)

create_lags(df_test, cols = ['전국_확진자수','전국_사망자수','전국_전일대비_증감수',
                        '제주_확진자수','제주_사망자수','제주_전일대비_증감수'],
            lags = [3,7,14,28], rolling_mean=True)


create_lags(df_test, cols=['평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '강수 계속시간(hr)', '10분 최다 강수량(mm)',
       '1시간 최다강수량(mm)', '일강수량(mm)', '최대 순간 풍속(m/s)', '최대 풍속(m/s)',
       '평균 풍속(m/s)', '평균 이슬점온도(°C)', '최소 상대습도(%)', '평균 상대습도(%)', '평균 증기압(hPa)',
       '평균 현지기압(hPa)', '최고 해면기압(hPa)', '최저 해면기압(hPa)', '평균 해면기압(hPa)',
       '합계 일조시간(hr)', '1시간 최다일사량(MJ/m2)', '합계 일사량(MJ/m2)', '평균 전운량(1/10)',
       '평균 중하층운량(1/10)', '합계 대형증발량(mm)', '합계 소형증발량(mm)', '안개 계속시간(hr)', '태풍'],
                lags = [1,3,5,7], rolling_mean=False)

In [379]:
df_test_columns = pd.DataFrame(df_test.columns)
df_test_columns.to_csv('df_test_columns.csv', index=False, encoding='cp949')

In [380]:
df_test.to_csv('df_test_ver0914.csv', encoding='cp949', index=False)

In [381]:
df_test.head().T

,0,1,2,3,4
기준일자,2021-07-01 00:00:00,2021-07-02 00:00:00,2021-07-03 00:00:00,2021-07-04 00:00:00,2021-07-05 00:00:00
행정동명,건입동,건입동,건입동,건입동,건입동
시_구분,제주시,제주시,제주시,제주시,제주시
행정동_코드,50110590,50110590,50110590,50110590,50110590
clust_방문거주,1,1,1,1,1
clust_방문거주2,2,2,2,2,2
kclus_배출,3,3,3,3,3
kclus_거주연령,2,2,2,2,2
kclus_시간유동,0,0,0,0,0
kclus_방문유동,0,0,0,0,0
